In [2]:
pip install "opendp[polars]"


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import opendp.prelude as dp
dp.enable_features("contrib")

# privacy_unit = dp.unit_of(contributions=1)
# input_metric, d_in = privacy_unit

In [4]:
import polars as pl
import pandas as pd 
data_url = "https://raw.githubusercontent.com/opendp/opendp/sydney/teacher_survey.csv"
# data = pl.scan_csv(data_url, ignore_errors=True)
data = pl.read_csv(data_url)
# data = pd.read_csv(data_url, header=None)
col_names = [
   "name", "sex", "age", "maritalStatus", "hasChildren", "highestEducationLevel", 
   "sourceOfStress", "smoker", "optimism", "lifeSatisfaction", "selfEsteem"
]
data.columns = col_names
df = pl.LazyFrame(data)
#cast all integers to floats
df = df.cast({pl.Int64: pl.Float64})
type(df)

polars.lazyframe.frame.LazyFrame

In [5]:
df.collect()

name,sex,age,maritalStatus,hasChildren,highestEducationLevel,sourceOfStress,smoker,optimism,lifeSatisfaction,selfEsteem
str,f64,f64,f64,f64,f64,str,str,str,f64,f64
"""Ariel Lucero""",2.0,21.0,1.0,2.0,5.0,"""1""","""2""","""19""",20.0,31.0
"""Sienna Shah""",2.0,42.0,4.0,1.0,2.0,"""5""","""2""","""23""",26.0,33.0
"""Denzel Crawfor…",2.0,47.0,4.0,1.0,2.0,"""4""","""2""","""24""",27.0,36.0
"""Emelia Norman""",2.0,48.0,4.0,2.0,3.0,"""3""","""2""","""30""",22.0,40.0
"""Abigail Koch""",2.0,41.0,7.0,1.0,6.0,"""1""","""2""","""21""",21.0,35.0
…,…,…,…,…,…,…,…,…,…,…
"""Luis Lara""",1.0,40.0,4.0,2.0,3.0,"""1""","""2""","""19""",17.0,30.0
"""Kasey Gray""",1.0,18.0,1.0,2.0,3.0,"""3""","""2""","""19""",25.0,38.0
"""Clarissa Frost…",2.0,39.0,7.0,1.0,5.0,"""3""","""2""","""23""",14.0,30.0


we specify the max num partitions
OpenDPException: 
  FailedFunction("max_num_partitions must be known when the metric is not sensitive to ordering (SymmetricDistance)")

During handling of the above exception, another exception occurred:
  MakeMeasurement("There is only one partition in select, so both max_influenced_partitions and max_num_partitions must either be unset or one")

During handling of the above exception, another exception occurred:

In [6]:
# import urllib.request
# data_url = "https://raw.githubusercontent.com/opendp/opendp/sydney/teacher_survey.csv"
# with urllib.request.urlopen(data_url) as data_req:
#     data = data_req.read().decode('utf-8')

context = dp.Context.compositor(
    data=df,
    privacy_unit= dp.unit_of(contributions=1),
    privacy_loss= dp.loss_of(epsilon=1.0),
    split_evenly_over=3, 
    margins={
        ("age", ): dp.Margin(public_info="lengths", max_partition_length=2000, max_num_partitions=1),
        (): dp.Margin(public_info="lengths", max_partition_length=2000, max_num_partitions=1),
    },
)

## Queries: count and mean 

In [7]:
context.query().select(
    pl.col("age").fill_null(40.0).fill_nan(40.0).dp.mean((0.0,100.0))
).release().collect()

age
f64
37.43892


In [8]:
df.select(
    pl.col("age").fill_null(40.0).fill_nan(40.0).mean()
).collect()

age
f64
37.43892


In [9]:
df.select(
    pl.col("age").fill_null(40.0).fill_nan(40.0).count()
).collect()

age
u32
6999


In [10]:
df.select(
    pl.col("age").fill_null(40.0).fill_nan(40.0).len()
).collect()

age
u32
6999


In [11]:
df.select(pl.len()).collect()

len
u32
6999


In [12]:
df.select(pl.all().len())

<LazyFrame [11 cols, {"name": UInt32 … "selfEsteem": UInt32}] at 0x17C2B17C0>

In [13]:
context.query().select(
   pl.all().len()
).release().collect()

ValueError: predicate always fails. An example traceback is shown above at 0.0.

In [ ]:
context.query().select(
    pl.col("age").fill_null(40.0).fill_nan(40.0).len().dp.noise()
).release().collect()

NameError: name 'context' is not defined

In [14]:
context.query().select(pl.col("age").fill_null(40.0).fill_nan(40.0).pl.len().dp.noise()
).release().collect()

AttributeError: 'Expr' object has no attribute 'pl'

In [27]:
context.query().filter(pl.col("age")>0).select(
    pl.len().dp.noise()
).release().collect()

len
u32
6997
